In [25]:
import numpy as np

# 给定的转移矩阵 A 和发射矩阵 B
A = np.array([[0.6, 0.3], 
              [0.4, 0.7]]).T
B = np.array([[0.8, 0.1], 
              [0.2, 0.9]]).T

# 观测序列编码，'up' 编码为 0，'down' 编码为 1
observations = np.array([0, 0, 1])
# observations = np.array([1, 1, 0])

# 初始概率分布 Pi
Pi = np.array([0.5, 0.5])


# 前向算法
def forward(obs_seq, Pi, A, B):
    N = len(obs_seq)
    S = A.shape[0]
    alpha = np.zeros((S, N))
    alpha[:, 0] = Pi * B[:, obs_seq[0]]

    for t in range(1, N):
        for s in range(S):
            alpha[s, t] = np.dot(alpha[:, t - 1], A[:, s]) * B[s, obs_seq[t]]
    return alpha


# 后向算法
def backward(obs_seq, A, B):
    N = len(obs_seq)
    S = A.shape[0]
    beta = np.zeros((S, N))
    beta[:, -1] = 1

    for t in range(N - 2, -1, -1):
        for s in range(S):
            beta[s, t] = np.sum(beta[:, t + 1] * A[s, :] * B[:, obs_seq[t + 1]])
    return beta


# 执行前向和后向算法
alpha = forward(observations, Pi, A, B)
beta = backward(observations, A, B)

# 计算 p(z1,z2|X, θ)
# 我们需要计算给定观测序列条件下的z1和z2的联合概率。这可以通过结合前向概率alpha和后向概率beta来实现。

# 计算 p(X) 为所有alpha在最后时间点的和 p(x1, x2, x3)
p_X = np.sum(alpha[:, -1])
print("alpha:")
print(alpha)
print("beta:")
print(beta)
print("p_X:")
print(p_X)

[0.02565  0.085725]
alpha:
[[0.4      0.204    0.02565 ]
 [0.05     0.0195   0.085725]]
beta:
[[0.258  0.48   1.    ]
 [0.1635 0.69   1.    ]]
p_X:
0.11137500000000002


In [26]:
# 对于每一对状态 (z1, z2) 计算联合概率
p_z1_z2_given_X = np.zeros((2, 2))
for z1 in range(2):
    for z2 in range(2):
        p_z1_z2_given_X[z1, z2] = (
            alpha[z1, 0] * A[z1, z2] * B[z2, observations[1]] * beta[z2, 1]
        )

# 归一化概率
p_z1_z2_given_X /= p_X

p_z1_z2_given_X

array([[0.82747475, 0.09912458],
       [0.05171717, 0.0216835 ]])

In [28]:
# To calculate P(z2, z3|X, θ), we will use the forward probabilities up to z2
# and the backward probabilities starting from z3, along with the transition
# and emission probabilities for z3. We do not need to consider z1 for this calculation.

# Initialize the joint probability matrix for z2 and z3
joint_prob_z2_z3 = np.zeros((2, 2))

# Calculate the joint probability for each combination of states for z2 and z3
for z2 in range(2):  # Possible states for z2
    for z3 in range(2):  # Possible states for z3
        # The joint probability is the product of:
        # - the forward probability of z2
        # - the transition probability from z2 to z3
        # - the emission probability of the observation at time 3 given z3
        # - the backward probability of z3
        joint_prob_z2_z3[z2, z3] = (
            alpha[z2, 1] * A[z2, z3] * B[z3, observations[2]] * beta[z3, 2]
        )

# We again normalize the joint probabilities by the total probability of the observations P(X)
conditional_prob_z2_z3 = joint_prob_z2_z3 / p_X

conditional_prob_z2_z3


array([[0.21979798, 0.65939394],
       [0.01050505, 0.11030303]])